# L-Shape domain

Let us solve the Poisson problem of finding $u$ satisfying 

$$
\begin{aligned}
-\Delta u = f && \text { in } \Omega,
\\
u  = 0 
&& \text{ on } \partial\Omega.
\end{aligned}
$$

#### 1. Import NGSolve and Netgen Python modules:

In [ ]:
from ngsolve import *
from ngsolve.webgui import Draw
from math import pi
from netgen.geom2d import SplineGeometry

#### 2. Polar coordinates in NGSolve

In [ ]:
r = sqrt(x**2+y**2)
phi = pi/2-IfPos(-y,IfPos(-x,atan2(x,y),IfPos(x,atan2(x,y),-pi)),atan2(x,y))
geoC = SplineGeometry()
geoC.AddCircle((0,0),1,bc="none")
meshC = Mesh(geoC.GenerateMesh(maxh=0.1))
Draw(r, meshC, "radius")
Draw(phi, meshC, "angle")

#### 3. Main discretization parameters and mesh

In [ ]:
order = 3
maxh = 0.2

R = 1
beta = 2/3 # no effect on geometry

# (Control-) Point positions
#  4        3        2
#           
#
#
#  5        0        1
#           
#           
#           
#  6        7
geo = SplineGeometry()
pnts = [ (0,0), (R,0), (R,R), (0,R), (-R,R), (-R,0), (-R,-R), (0,-R) ]
pind = [ geo.AppendPoint(*pnt) for pnt in pnts ]

# line from pind[0] to pind[1]
geo.Append(['line',pind[0],pind[1]],bc="outer")
# spline from point pind[1] to pind[3] with control point pind[2]
geo.Append(['spline3',pind[1],pind[2],pind[3]],bc="outer")
geo.Append(['spline3',pind[3],pind[4],pind[5]],bc="outer")
geo.Append(['spline3',pind[5],pind[6],pind[7]],bc="outer")
geo.Append(['line',pind[7],pind[0]],bc="outer")

mesh = Mesh(geo.GenerateMesh(maxh=maxh))
mesh.Curve(order)

Draw(mesh)

#### 4. Reference solution and corresponding right hand side


In [ ]:
ex_sol = (R**2-r**2)*r**beta*sin(phi*beta)
rhs = (4*beta+4)*r**beta*sin(phi*beta)

ex_sol_grad = CoefficientFunction( (ex_sol.Diff(x),ex_sol.Diff(y)) )
#ex_sol_grad = ((R**2-r**2)*beta*r**(beta-1)*sin(phi*beta)-2*r**(beta+1)*sin(phi*beta))*CoefficientFunction( (cos(phi),sin(phi)) ) + beta*(R**2-r**2)*r**(beta-1)*cos(phi*beta)*CoefficientFunction( (-sin(phi),cos(phi)) )

# Regularize singularity at (0,0) to enable drawing
r_eps = sqrt((x+1e-8)**2+(y+1e-8)**2)
grad_eps = ((R**2-r_eps**2)*beta*r_eps**(beta-1)*sin(phi*beta)-2*r_eps**(beta+1)*sin(phi*beta))*CoefficientFunction( (cos(phi),sin(phi)) ) + beta*(R**2-r_eps**2)*r_eps**(beta-1)*cos(phi*beta)*CoefficientFunction( (-sin(phi),cos(phi)) )

sol_grad = IfPos(r, ex_sol_grad, grad_eps)
Draw(ex_sol, mesh, "ex_solution")
Draw(Norm(sol_grad), mesh, "ex_solution_grad", deformation=True)

#### 5. Declare a finite element space and gridfunction

In [ ]:
fes = H1(mesh, order=order, dirichlet="outer")
sol = GridFunction(fes)

#### 6. Define and assemble linear and bilinear forms

In [ ]:
u,v = fes.TnT()

a = BilinearForm(fes, symmetric=True)
a += grad(u)*grad(v)*dx
a.Assemble()

f = LinearForm(fes)
f += rhs*v*dx
f.Assemble()

#### 7. Solve system and draw solution

In [ ]:
sol.vec.data = a.mat.Inverse(freedofs=fes.FreeDofs()) * f.vec
Draw(sol, mesh, "sol")
Draw(Norm(Grad(sol)), mesh, "sol_grad", deformation=True)

#### 8. Compute error

In [ ]:
H1err = sqrt( Integrate((ex_sol-sol)**2 + InnerProduct(ex_sol_grad-Grad(sol),ex_sol_grad-Grad(sol)), mesh) )
print("err = ", H1err)
Draw((ex_sol-sol)**2 + InnerProduct(sol_grad-Grad(sol),sol_grad-Grad(sol)),mesh, "err")